# Planner

When we know exactly which semantic function we want to invoke in our app, we can just specify it in our code. As we grow our library of interesting and useful semantic functions, we might not want a specific button in our UI to invoke each function. Wouldn't it be cool if the AI could determine which semantic function best fits the user's question?

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230704.3-preview"

#!import ../config/SettingsHelper.cs

using Microsoft.SemanticKernel;

MySettings settings = Settings.LoadFromFile();

// Configure AI backend used by the kernel
KernelBuilder builder = new();
if (settings.Type == "azure")
{
    if (!string.IsNullOrWhiteSpace(settings.AzureOpenAI.ChatDeployment))
        builder.WithAzureChatCompletionService(settings.AzureOpenAI.ChatDeployment, settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
    if (!string.IsNullOrWhiteSpace(settings.AzureOpenAI.CompletionsDeployment))
        builder.WithAzureTextCompletionService(settings.AzureOpenAI.CompletionsDeployment, settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
}
else
    builder.WithOpenAIChatCompletionService(settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
IKernel kernel = builder.Build();

# Action Planner

The Action Planner chooses one - and only one - semantic function from the list of functions you provide to the kernel.

Before you run the following code, open the CelebratePlugin.cs file in the Plugins folder. Here, you'll see more than one SKFunction are defined (e.g. GenerateOccasionCard and GenerateGiftIdeas).

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;

#!import Plugins/CelebratePlugin.cs

var celebratePlugin = kernel.ImportSkill(new CelebratePlugin(kernel));

ActionPlanner planner = new(kernel);
Plan plan = await planner.CreatePlanAsync("I have no idea what to get my dad for father's day!");

Console.WriteLine(plan.ToJson(true));

SKContext result = await plan.InvokeAsync();

Console.WriteLine(result);

I know the response is long, but I want you to see the plan generated.

The response you are looking for is all the way at the bottom where it generated a list of ideas for father's day gifts even though we didn't tell the kernel to invoke the GenerateGiftIdeas function. There are two AI calls happening here:
1. We pass the prompt and a list of our available plugins/function to an instance of ActionPlanner. The ActionPlanner takes care of passing descriptions of each of our funtions to AI and determine which best fits the user's provided input.
2. The second call passes our Father's Day prompt to the GenerateGiftIdeas function - and no other function.

# Sequential Planner



In [ ]:
using Microsoft.SemanticKernel.Orchestration;

// register plugins for getting thematic travel destinations, hotel availability, rental car availability, points of interest, and flight availability
string travelDestinations = "give me a list of travel destinations";
var travelDestinationsFunction = kernel.CreateSemanticFunction(travelDestinations);

string hotelAvailability = "give me hotel availability for these destinations: {{$destinations}}";
var hotelAvailabilityFunction = kernel.CreateSemanticFunction(travelDestinations);

string travelDestinations = "give me a list of travel destinations";
var summaryFunction = kernel.CreateSemanticFunction(travelDestinations);

// user's prompt: book a trip
string destination = "I want to book a two-week trip to somewhere warm and sunny.";

// create an instance of a planner and provide a connection to the kernel
SequentialPlanner planner = new(kernel);

Console.WriteLine(completion);

# Exercise

